# Hands-on with the BLACK satisfiability checker

Welcome to our AAAI-23 lab session on BLACK (https://www.black-sat.org), a satisfiability checker and temporal reasoning tool for *Linear&nbsp;Temporal&nbsp;Logic*&nbsp;(LTL) and related formalisms.

This Colab Notebook will guide you through the activities of this lab, introducing to the usage of BLACK's tool and its Python API.

But first... we need to install BLACK in the Notebook's environment. 
To do it, run the following code block before proceeding.

In [ ]:
!echo -n Installing BLACK... && wget -q https://www.inf.unibz.it/~gigante/aaai23-black/black-sat-0.9.2-Linux.deb > /dev/null && apt-get update >/dev/null && apt-get install -y ./black-sat-0.9.2-Linux.deb >/dev/null && echo " Done!"

Then, if everything worked correctly, the following snippet should be able to import BLACK's Python module.

In [ ]:
from black_sat import *

Everything's alright, so let's start!

# Introduction



BLACK can be used as a command line tool, or as a *C++* or *Python* API to be embedded in client applications. Here we will learn how to use the command-line interface first, and then we will explore the Python API.

Let's first introduce basic concepts.

LTL is a formalism to talk about the *truth of facts evolving over time*. 

An example:

$$ p \land \mathsf{G} q \land \mathsf{F}\neg p$$

This formula describes the evolution of two Boolean facts ($p$ and $q$), over an infinite sequence of *time steps*, or *states*.

It says that:

1.   At the first step, $p$ must be *true*
2.   in every step of the sequence ($\mathsf{G}$, for *globally*), $q$ must hold ($\mathsf{G}q$)
3.   in some *future* step of the sequence, $p$ must be *false* ($\mathsf{F}\neg p$)

Now we can ask ourselves some questions:
1.   Is there such a sequence, that is, is $\phi$ *satisfiable*?
2.   If $\phi$ is satisfiable, how does such a sequence look like?

We can ask BLACK to answer for us!


# Command-line interface

BLACK can be asked for the satisfiability of an LTL formula from the command line. 

We can run shell commands in this Notebook by prefixing the commands with `!`.

## Basic usage

The basic usage is very simple:

In [ ]:
!black solve -f 'p & G(q) & F(!p)'

BLACK answers `SAT`, meaning that the formula is satisfiable, *i.e.*, there is a sequence (a *model*) satisfying it. 

The `-f` option tells BLACK that we are giving it the formula from the command line. Without it, it would look for a file name to read the formula from. 

## Model extraction

So a model for the formula exists. How does such a model look like? We can ask BLACK to produce one with the `-m` option:

In [ ]:
!black solve -m -f 'p & G(q) & F(!p)'

BLACK produces an infinite model, which is however represented finitely by two states, with the second one repeating infinitely. 

The produced model is one where:
1.   In the first step, $p$ and $q$ hold. 
2.   In the second step, and in all the infinite subsequent ones, $q$ holds but $p$ does not. 



## Unsatisfiable formulas

Not every formula can be satisfied. Contradictions can result into *unsatisfiable* formulas. For example:

$$ \mathsf{G}(p) \land \mathsf{F}(\neg p) $$

This formula is asking for a sequence where:
1.   $p$ always holds, and
2.   $p$ will eventually be false

These are of course contradictory requirements. So BLACK cannot find a model for the formula:

In [ ]:
!black solve -f 'G p & F!p'

We'll see later that BLACK can tell us more about unsatisfiable formulas.

## Other LTL operators

LTL has many other *temporal operators* besides $\mathsf{F}$ and $\mathsf{G}$. The two most important are:
1.   The *tomorrow* operator (*e.g.* $\mathsf{X}p$) that says that something ($p$) has to happen at the *next* time step.
2.   The *until* operator (*e.g.* $p \mathbin{\mathsf{U}} q$), a binary operator that says that:
      1. its right operand ($q$) will hold in the future, and
      2. the left operand ($p$) will continuously hold until that point (excluded)

An example formula using those operators is the following:

$$ p \land \mathsf{X} p \land (q \mathbin{\mathsf{U}} \neg p)$$

Here we are asking $p$ to hold at the first and second state of the sequence, and $\neg p$ to hold eventually. In every step before $\neg p$, $q$ must hold.

Let's see which model is found by BLACK:

In [ ]:
!black solve -m -f 'p & X p & (q U !p)'

As expected, the first two states contain both $p$ and $q$, while the third contains $\neg p$, and is the one that is repeated infinitely for the rest of the model. 

Note that BLACK introduced $\neg p$ at the first available time step. That's because BLACK always finds the *shortest* models.

## Past operators

Sometimes, when planning for the future, it is useful to take inspiration from the past. BLACK supports LTL *past operators*, that are the past counterparts of the *future operators* that we used until now.

An example:

$$ \mathsf{F}( p_1 \to \mathsf{YO} q_1 \land p_2 \to \mathsf{YO} q_2)$$

Here the *once* operator ($\mathsf{O}$) is the past dual of $\mathsf{F}$, and the *yesterday* operator ($\mathsf{Y}$) is the past dual of $\mathsf{X}$. 

This formulas says that there is some point in the future, where:
1.   if $p_1$ holds, we have seen a $q_1$ before, and
2.   if $p_2$ holds, we have seen a $q_2$ before.

Let's see what BLACK says about it:

In [ ]:
!black solve -m -f 'F((p_1 -> Y O(q_1)) & (p_2 -> Y O(q_2)))'

That's fine. BLACK found the shortest model, which indeed is the one where neither $p_1$ nor $p_2$ ever hold. Let's try to force at least one of them to happen:

In [ ]:
!black solve -m -f 'F((p_1 | p_2) & (p_1 -> Y O(q_1)) & (p_2 -> Y O(q_2)))'

Nice. Of the two possibilities, BLACK chose to realize $p_2$, and therefore $q_2$ before that.

Note that writing formulas equivalent to these using *future* operators is possible, but very cumbersome, because we do not know in which *order* $q_1$ and $q_2$ have to happen. 

So the formula equivalent to the first above using future operators is much bigger: 

$$ \mathsf{F}(\neg p_1 \land \neg p_2) \lor \mathsf{F}(q_2 \land \mathsf{X} \mathsf{F} \neg p_1) \lor \mathsf{F}(q_1 \land \mathsf{X} \mathsf{F} \neg p_2) \lor (\mathsf{F}q_1 \land \mathsf{F}q_2) $$

## Finite traces

In the AI community, LTL is often interpreted over *finite traces*, and called LTLf in this case.

LTLf has the same syntax of LTL but the models are finite sequences.

This often changes the meaning of formulas. For example:

$$ \mathsf{G}(\mathsf{X} p) $$

This formula is equivalent to $\mathsf{X}(\mathsf{G} p)$ in LTL, which is satisfiable, but it is *unsatisfiable* in LTLf, because the *tomorrow* operator *requires* the existence of a successor state (where $q$ holds, in this case), but this cannot happen at every state of the sequence because the last state does not have a successor!

We can ask BLACK to interpret a formula over finite traces with the `--finite` option, and compare the two semantics:

In [ ]:
!black solve -f 'G(X p)'

In [ ]:
!black solve --finite -f 'G(X p)'

Often, in LTLf, we need to express the fact that something has to happen at the next state, but only *if it exists*. 

This is done with the *weak tomorrow* operator ($\mathsf{\widetilde{X}}$), as in:

$$ \mathsf{G}(\mathsf{\widetilde{X}} p) $$

which is satisfiable in LTLf (and is equivalent to $\widetilde{\mathsf{X}}(\mathsf{G} p)$). BLACK confirms:

In [ ]:
!black solve --finite -f 'G(wX p)'

## Equivalence and validity

If a formula is *unsatisfiable*, its negation is *valid*, that is, satisfied by *any model*. 

A formula can be checked for validity in BLACK by simply checking its negation. 

This is useful, among other things, to check *entailment* or *equivalence* between formulas. 

For example, $p\mathbin{\mathsf{U}} p$ is equivalent to $p$, and we can check it as follows:

In [ ]:
!black solve -f '!(p <-> p U p)'

We can also check the equivalences mentioned in previous sections:

In [ ]:
!black solve -f '!(G X p <-> X G p)'

In [ ]:
!black solve --finite -f '!(G wX p <-> wX G p)'

In [ ]:
!black solve -f '!(F((p_1 -> Y O(q_1)) & (p_2 -> Y O(q_2))) <-> (F(!p_1 & !p_2) | F(q_2 & X F !p_1) | F(q_1 && X F !p_2) | (F q_1 && F q_2)))'

## Unsatisfiable cores

Consider this formula:

$$ \mathsf{G}(p \land (q \lor r \to w)) \land \mathsf{F}(\neg p \land (\neg q \land r \land w)) $$

It seems complex. Let's start by asking BLACK if it is satisfiable:

In [ ]:
!black solve -f 'G(p & (q | r -> w)) & F(!p & (!q & r & w))'

It is unsatisfiable indeed.

But, suppose this is a specification for a system you are designing, or the encoding of a *planning* problem that turns out to not have any solution.

In these cases, you probably need to know *why* the formula is unsatisfiable. 

This further question can be often answered by extracting an *unsatisfiable core* for the formula, by using the `-c` option of BLACK:

In [ ]:
!black solve -c -f 'G(p & (q | r -> w)) & F(!p & (!q & r & w))'

BLACK this time gave a more detailed output.

The produced formula is an *unsatisfiable core* of the original one.

It means that:
1.  some subformulas ($q\lor r\to w$ and $\neg q\land r\land w$) have been *replaced* by placeholders (`{0}` and `{1}`), but
2.  the formula is still *unsatisfiable*.

Indeed we can check the new formula is unsatisfiable with BLACK (the double braces are to escape them in Colab):

In [ ]:
!black solve -f 'G(p & {{1}}) & F(!p & {{2}})'



This means the removed subformulas are not involved with the unsatisfiability and can be ignored.

Indeed, the formula is much simpler now and why it is unsatisfiable is evident: we are requiring $p$ to always hold ($\mathsf{G}(p)$) but also to be somewhere false ($\mathsf{F}(\neg p)$).

BLACK finds always a *minimal unsatisfiable core*, that is, one that cannot be reduced further.

## JSON output

The above output is humanly readable, but if we had to pass BLACK's output to another piece of software for further processing, it would be hard to parse.

For this reason, BLACK supports multiple output formats. The current supported ones are `readable` (the default, shown above) and `json`. You can choose the output format with the `-o` option:

In [ ]:
!black solve -o json -m -f 'p & G(q) & F(!p)'

The JSON output is available also when we ask for the *unsatisfiable cores*:

In [ ]:
!black solve -o json -c -f 'G(p & (q | r -> w)) & F(!p & (!q & r & w))'

## Trace checking

Thanks to the JSON output, BLACK can *read back* its own output to check its correctness. 

The `black check` command can read the output of the `black solve` command and check if the produced model is correct for the given formula. This is handy to do with a shell pipe.

For example:

In [ ]:
!black solve -o json -m -f 'p & X p & (q U !p)' | black check -t - -f 'p & X p & (q U !p)'

The `-t` option tells the second BLACK command where to find the trace (the model) to check. 

In this case, `-t -` means to read from the standard input (that comes from the pipe).

The output should always be `TRUE`, meaning that the produced model is indeed correct for the formula. 

A `FALSE` output would mean we spotted a bug in the solver!

## Playground

The following form calls BLACK's command line interface with all the options seen so far. Have fun!

In [ ]:
#@title  { run: "auto" }
#@markdown BLACK command-line playground
formula = "!( (p U q) \u003C-> F(wX False & O(q & Z H p)) )" #@param {type:"string"}
finite = True #@param {type:"boolean"}
model = True #@param {type:"boolean"}
core = True #@param {type:"boolean"}
json = False #@param {type:"boolean"}
check_trace = False #@param {type:"boolean"}

!black solve \
  {"-m" if model or check_trace else ""} \
  {"--finite" if finite else ""} \
  {"-c" if core else ""} \
  {"-o json" if json or check_trace else ""} \
  -f '{formula}' \
  {("| black check -t - -f '" + formula + "' " + ("--finite" if finite else "")) if check_trace else ""}





# Python API

The command-line tool we explored so far is only a thin wrapper over BLACK's C++ library, which provides a comprehensive API for creating, manipulating and reasoning about temporal logic formulas. 

Here we will explore the basics of BLACK's API through its Python interface. 

Doing so, we will also explore some features of BLACK that are not fully available through the command-line interface (yet).

**Warning**: the Python bindings are work-in-progress and yet unreleased, so you are getting a live preview!

## Creating formulas

A formula is the central kind of objects in BLACK's API. To create one, we need first to create its *alphabet*, that is, the collection of the symbols the formula talks about. 

This is done by instantiating an object of type `alphabet`. Since in formal texts the alphabet is usually called $\Sigma$, we will call the `alphabet` object `sigma`.

In [ ]:
sigma = alphabet()

Before constructing a formula, we have to ask the alphabet for the symbols that will be used in the formula. 

For an LTL formulas, the basic symbols are `proposition`s.

In [ ]:
p = sigma.proposition("p")
q = sigma.proposition("q")

Now we can finally construct a formula. BLACK's Python module overloads the majority of operators to make the formula construction syntax as smooth as possible. 

For example:

In [ ]:
f = p & G(q) & F(~p)
print(f)

Formulas can also be *parsed* from strings, that can come from text files or from anywhere.

This can be done with the `parse_formula` function, which accepts the `alphabet` object and a string, and returns the parsed formula, or `None` if some syntax error was found:

In [ ]:
g = parse_formula(sigma, "p & G q & F !p")
assert g is not None
print(g)

## Manipulating formulas

Formulas can be compared for equality in BLACK's API.

The comparison is:
1.   **syntactic**: the syntax of the formulas is compared, not the identity of the objects
2.   **fast**: the comparison does not recursively descends the formulas. Under the hood, equal formulas are represented by the same object, so we just compare pointers

So we can confirm that `f` and `g` are actually the same formula:

In [ ]:
assert f == g

Now that we have a formula, we can inspect it.

Each different logical or temporal operator corresponds to a different object type. 

In this case, `f` is of type `conjunction`, and we can access its left and right operands with aptly named properties:

In [ ]:
assert isinstance(f, conjunction)
print(f"left operand: {f.left}")
print(f"right operand: {f.right}")

The right operand consists of two nested unary operators (`F` and `!`).

So, for example, to get the `proposition` inside them, we can do:

In [ ]:
print(f.right.argument.argument)

With these building blocks we can manipulate the formulas in many ways.

Indeed, the whole BLACK solver is built by manipulating formulas with (the C++ version of) this API.

## Invoking the solver

Once we have the formula, we may want to check its satisfiability.

For that, we need an instance of the `solver` class.

In [ ]:
slv = solver()

To solve the formula we can call the `solve` method of `solver`.

It accepts:
1.   an object of type `scope`. Let's set it aside for now, we'll understand later what its purpose is;
2.   the formula to solve
3.   an optional `finite` parameter of type `Bool` that tells whether we want to interpret the formula over finite (`True`) or infinite (`False`) traces.

For example, suppose we want to solve `f` over finite traces:

In [ ]:
xi = scope(sigma)
slv.solve(xi, f, True)

## Inspecting the model

Now that the solver found that the formula is satisfiable, we can access its model. 

This is done with the `model` property and its `size` and `value()` accessors.

In [ ]:
print(f"Model size: {slv.model.size}")

for t in range(slv.model.size):
  for prop in [p,q]:
    print(f"{prop} at t = {t} is {slv.model.value(prop, t)}")

## LTL modulo theories

Sometimes, propositions are not enough to model a given property or system.

In these cases, BLACK can deal with an extension of LTL called *LTL modulo theories*, where propositions are replaced by general *first-order sentences* interpreted over arbitrary theory, *à la* SMT.

Let's see how these formulas can be built.

### Creating formulas

To start, we need some *variables*. Let's ask for them to the alphabet.

In [ ]:
x = sigma.variable("x")
y = sigma.variable("y")

Instantiating the variables in not enough, we need also to decide their *sort* (or *type*). 

Let's say these two variables are integers. We declare them as such to the `scope` object we created before.

In [ ]:
xi.declare(x, sigma.integer_sort())
xi.declare(y, sigma.integer_sort())

Now let's say we want both `x` and `y` to start at zero, and we want them to reach a value greater than 10 while `x` remains always the double of `y`.

Such a formula looks like this:

$$ x = 0 \land y = 0 \land \mathsf{G}(x = 2y) \land \mathsf{F}(y > 10) $$

We can build it with BLACK's API as follows:

In [ ]:
f = (x == 0) & (y == 0) & G(x == 2 * y) & F(y > 10)
print(f)

### Look-aheads

Now, what if we want the variables to *increase* at each step?

Then we have to tie their value at each time to their value at the next one.

This can be done with the *look-ahead* ($\mathsf{next}$) and *weak look-ahead* ($\mathsf{wnext}$), which are used as:

$$ x = 0 \land y = 0 \land \mathsf{G}(\mathsf{wnext}(x) > x \land x = 2y) \land \mathsf{F}(y > 10) $$

Note that LTL modulo theories formulas are always interpreted over *finite traces*, hence the difference between the strong and weak look-aheads, much like the difference between the strong and weak *tomorrow* operators that we have seen before. 

The new formulas can be built as follows:

In [ ]:
f = (x == 0) & (y == 0) & G((wnext(x) > x) & (x == 2*y)) & F(y > 10)
print(f)

### Solving the formulas

To solve this new formula, we proceed as before. But this time, it is better to first check for the *type correctness* of the formula. 

We can ask the `scope` to perform this check:

In [ ]:
def report(str):
  print(f"Type error: {str}")

assert xi.type_check(f, report)

So everything's alright, but before proceeding, we have to pay attention. 

LTL modulo theories is in general *undecidable*, and only *semi-decidable*, so BLACK can reliably terminate only on *satisfiable* instances. 

If we come across an unsatisfiable instance and the solving takes too much time, we can stop the computation using the Notebook's stop button.

That said, the solver can be invoked as always, ensuring we ask for the *finite-trace* semantics:

In [ ]:
slv.solve(xi, f, True)

The formula is satisfiable indeed, so we can access its model.



In [ ]:
assert slv.model is not None
slv.model.size

It is not possible to directly extract the value of the variables from the model (yet, stay tuned), but it is possible to test the truth of any predicate.

So for example, we may ask whether $x$ is actually the double of $y$ at time step $1$.

In [ ]:
slv.model.value(x == 2 * y, 1)

### Uninterpreted predicates

LTL modulo theories supports integer and real arithmetic, also combined with *uninterpreted predicates*, that is, $n$-ary predicates over arbitrary sorts that do not have a predefined meaning (in contrast to, say, the $<$ predicate, which is always interpreted as the arithmetic comparison).

To declare a predicate we can ask for its symbol from the `alphabet`, and then declare its arity and domain to the `scope`:

In [ ]:
r = sigma.relation("r")

integer = sigma.integer_sort()
xi.declare(r, [integer, integer, integer], scope.rigid)

Here we have declared a ternary predicate $r$ over integer numbers. 

The `scope.rigid` option means that the predicate does *not* evolve in time.

Now, we want to model the fact if $r(x, y, z)$ holds, then $z$ is included in the interval $(x,y)$.

This can be done as:

$$ \forall x : \mathsf{Int}, y : \mathsf{Int}, z : \mathsf{Int} \,\, r(x, y, z) \to (x < z \land z < y)$$

and we can build this formula in BLACK as follows:

In [ ]:
z = sigma.variable("z")

f = forall([x[integer], y[integer], z[integer]], implies(r(x, y, z), (x < z) & (z < y)))
print(f)

This is kind of an axiom for our $r$ relation. The formula itself is trivial to satisfy, by negating the antecedent of the implication. 

Rather, the interesting question is whether BLACK can enforce the axiom.

So let's see if the formula is consistent with the fact $r(3, 6, 10)$ (spoiler: it's not):

In [ ]:
print(slv.solve(xi, f & r(3, 6, 10), True))

Indeed. Now we can combine this axiom with any temporal operator of LTL modulo theories. For example:

In [ ]:
g = (x == 0) & (y == 2) & G(r(x, y, z) & (wnext(z) == z + 1)) & F(z == 10)
print(g)

This initializes $x=0$ and $y=2$, and increments the variable $z$ at each step, while requiring $r(x,y,z)$ to hold, stopping when $z = 10$. 

Hence, to satisfy the above axiom, $z$ should start at $1$, and by the end of the trace, $y$ should be at least $11$. 

Let's see if BLACK agrees.

In [ ]:
xi.declare(x, integer)
xi.declare(y, integer)
xi.declare(z, integer)

assert xi.type_check(f & g, report)

assert slv.solve(xi, f & g, True)
assert slv.model is not None

last = slv.model.size - 1
print(f"Model size: {slv.model.size}")
print(f"z == 1 at t = 0: {slv.model.value(z == 1, 0)}")
print(f"y >= 11 at t = {last}: {slv.model.value(y >= 11, last)}")

Everything as expected!

# Conclusions

So we explored many functionalities of BLACK and its APIs.

BLACK may be not as full-featured as nuXmv or SPOT, but is growing and taking up its space in the temporal reasoning landscape.

A few things are missing from this overview:
1.   uninterpreted and enumerated sorts in LTL modulo theories
2.   more formulas manipulation facilities
3.   the SAT/SMT solvers backend abstraction layer
4.   the C++ API

For any question, even after the AAAI lab will be ended, feel free to write to us.

Enjoy the rest of the conference!

Nicola and Luca